Copyright 2021 National Technology & Engineering Solutions<br>
of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,<br>
the U.S. Government retains certain rights in this software.<br>
<br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>
<br>
   http://www.apache.org/licenses/LICENSE-2.0<br>
<br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator

In [4]:
logging.basicConfig(level=logging.INFO)

In this example we'll be generating a small matix of secret shared values and then raising them to a public power (in this case 4). You'll see that the computation is correct when the result is revealed for both the positive and negative bases.

Feel free to alter the parameters in the next cell to change the behavior if you have cases you'd like to try.

In [8]:
testvec = numpy.array([-1,2,3,4], dtype=object).reshape(2,2)    
powval = 4

In [9]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()

    secret_share = protocol.share(src=0, secret=protocol.encoder.encode(testvec), shape=(2,2))
    secret = protocol.encoder.decode(protocol.reveal(secret_share))
    log.info(f"Player {communicator.rank} secret: \n{secret}")
    exp_share = protocol.exp(secret_share,powval) 
    exp = protocol.encoder.decode(protocol.reveal(exp_share))
    log.info(f"Player {communicator.rank} exp: \n{exp}")
    log.info(f"Player {communicator.rank} expected: \n{testvec**powval}")
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:55771 from tcp://127.0.0.1:55771.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:55771 from tcp://127.0.0.1:55772.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:55771 from tcp://127.0.0.1:55773.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret: 
[[-1.  2.]
 [ 3.  4.]]
INFO:root:Player 1 secret: 
[[-1.  2.]
 [ 3.  4.]]
INFO:root:Player 2 secret: 
[[-1.  2.]
 [ 3.  4.]]
INFO:root:Player 0 exp: 
[[  1.  16.]
 [ 81. 256.]]
INFO:root:Player 1 exp: 
[[  1.  16.]
 [ 81. 256.]]
INFO:root:Player 2 exp: 
[[  1.  16.]
 [ 81. 256.]]
INFO:root:Player 0 expected: 
[[1 16]
 [81 256]]
INFO:root:Player 1 expected: 
[[1 16]
 [81 256]]
INFO:root:Player 2 expected: 
[[1 16]
 [81 256]]
INFO:cicada.communi

[None, None, None]